# Using the Airtable API in Python

In the terminal, run  
`pip install pyairtable`. 

Find your API key on the Airtable website ([here](https://airtable.com/account)), and save it as an environment variable under the name `AIRTABLE_API_KEY`. API keys are account-dependent, and not table-dependent. 

In [7]:
import os
api_key = os.environ["AIRTABLE_API_KEY"]

An overview of the API specific to our base (as generated by Airtable) can be found [here](https://airtable.com/apptUkOtU3W8HxbQQ/api/docs#curl/authentication). This is also where you will find the value of `base_id` that was used below. 

In [32]:
from pyairtable import Table
import pandas as pd
base_id = "apptUkOtU3W8HxbQQ"
table_name = "Company List"

In [33]:
table = Table(api_key, base_id, table_name)

For now, we'll just load all the data in one go. We could also query individual records, or sets of records.

In [17]:
all_data = table.all()
len(all_data)

1206

In [21]:
all_data[0]

{'id': 'rec00MDWnik2I0nA8',
 'fields': {'Web': 'https://softsmile.com/',
  'Original Source': 'TechNYC',
  'Owner': {'id': 'usr9BPdDIhuQuzTkj',
   'email': 'atownsend@cornell.edu',
   'name': 'Anthony Townsend'},
  'Urban Tech Screening': 'Failed',
  'Sniff Test Notes': 'dental software\n',
  'Company': 'SoftSmile'},
 'createdTime': '2022-01-05T20:22:37.000Z'}

We can see that this is a nested dictionary. For ease of use, we will now flatten it, and load it into a `DataFrame`.

In [25]:
df = pd.json_normalize(all_data, sep='_')
df.head()

,id,createdTime,fields_Web,fields_Original Source,fields_Owner_id,fields_Owner_email,fields_Owner_name,fields_Urban Tech Screening,fields_Sniff Test Notes,fields_Company,...,fields_Sub-category,fields_Confidence,fields_How big of a scale is it?,fields_Metadata Sources + Notes,fields_Year Acquired or Dissolved,fields_Valuation at Acquisition,fields_Revenue/Budget,fields_Keywords,fields_Contact Email,fields_Contact Name
0,rec00MDWnik2I0nA8,2022-01-05T20:22:37.000Z,https://softsmile.com/,TechNYC,usr9BPdDIhuQuzTkj,atownsend@cornell.edu,Anthony Townsend,Failed,dental software\n,SoftSmile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rec019MVAabtvqZ29,2022-01-05T20:22:37.000Z,https://www.loop.ai/,TechNYC,usr9BPdDIhuQuzTkj,atownsend@cornell.edu,Anthony Townsend,Failed,looks like AI consultants\n,Loop AI Labs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rec020xb3hcLqlI4e,2022-01-05T20:22:37.000Z,https://www.yapp.us/,TechNYC,usr9BPdDIhuQuzTkj,atownsend@cornell.edu,Anthony Townsend,Failed,movile apps\n,Yapp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rec0C1X6J53tsmZHo,2021-10-26T16:28:52.000Z,https://navigine.com,NaN,usrzlz6zDGzjGmaEg,mfd64@cornell.edu,Max Dumas,Passed,Indoor navigation and way-finding systems\n,Navigine,...,[Mobility Systems: Navigation & Wayfinding],3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rec0CxUiuuPRFVKNw,2022-01-05T20:22:37.000Z,https://horizon-ventures.net/,TechNYC,usr9BPdDIhuQuzTkj,atownsend@cornell.edu,Anthony Townsend,Failed,NaN,Horizon Ventures,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can now use this DataFrame for a set of queries. 

In [27]:
df.columns

Index(['id', 'createdTime', 'fields_Web', 'fields_Original Source',
       'fields_Owner_id', 'fields_Owner_email', 'fields_Owner_name',
       'fields_Urban Tech Screening', 'fields_Sniff Test Notes',
       'fields_Company', 'fields_Total Investment',
       'fields_Does it involve government?', 'fields_Number of Employees',
       'fields_Business Address', 'fields_Employees', 'fields_Year Founded',
       'fields_Location', 'fields_categorization notes', 'fields_Category',
       'fields_How urban is it?', 'fields_Sub-category', 'fields_Confidence',
       'fields_How big of a scale is it?', 'fields_Metadata Sources + Notes',
       'fields_Year Acquired or Dissolved', 'fields_Valuation at Acquisition',
       'fields_Revenue/Budget', 'fields_Keywords', 'fields_Contact Email',
       'fields_Contact Name'],
      dtype='object')

In [31]:
df_passed = df[df['fields_Urban Tech Screening'] == "Passed"]
df_passed = df_passed.reset_index()
df_passed.shape

(265, 31)

So 265 companies passed our screening. 

The [pyAirtable readTheDocs](https://pyairtable.readthedocs.io/en/latest/api.html#overview) specifies other methods that can be used. 